In [3]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import AdvancedBraggEdgeFitting_v2
from PIL import Image



In [4]:
img = Image.open('/media/carminati_c/Data2/samples_for_Chiara_LSP.tif')

In [5]:
plt.figure()
plt.imshow(img[:,:,0])

TypeError: 'TiffImageFile' object is not subscriptable

<Figure size 432x288 with 0 Axes>